In [10]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [11]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + c + '-sell.csv'
    str_spl = 'MS-' + letter + c + '-supply.csv'
    str_d = 'MS-' + letter + c + '-daily.csv'
    return str_i, str_sl, str_spl, str_d

Количество магазинов в штате

In [12]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

In [40]:
def get_files(state, path):
    max = num_shop(state)
    files = {}
    path = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(path + state + c + '*.csv')
        print(path[shop])
        files[shop] = {}
        for j in range(0, len(path[shop])):
            files[shop][j]= pd.read_csv(path[shop][j])
        shop = shop + 1
    return files, max

In [47]:
sold = {}
def daily_stor(state, id):
    
    files, max = get_files(state, './input/*')
    answer = {}
    
    INV = 0
    SELL = 1
    SPL = 2
    DAILY = 3
    
    shop = 0
    while shop < max:
        
        #print(files[shop][INV])
        #print(files[shop][SPL])
        files[shop][SELL]['date'] = pd.to_datetime(files[shop][SELL]['date'], format='%Y-%m-%d')
        files[shop][INV]['date'] = pd.to_datetime(files[shop][INV]['date'], format='%Y-%m-%d')
        files[shop][SPL]['date'] = pd.to_datetime(files[shop][SPL]['date'], format='%Y-%m-%d')
        
        files[shop][SELL]['sku_num'] = files[shop][SELL]['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
        daily = pd.crosstab(files[shop][SELL]['date'], files[shop][SELL]['sku_num'], margins = True)
        print(daily)
        daily = daily.drop(['All'])
        daily_new = pd.DataFrame(columns=['date', 'apple', 'pen'])
        daily_new['date'] = files[shop][SELL]['date'].drop_duplicates()
        daily_new['apple'] = daily[0].values
        daily_new['pen'] = daily[1].values
        daily_new.index = range(0, daily[0].size)
        
        sold[id] += daily_new
        print(daily_new.head())
        
        daily_new.index = files[shop][SELL]['date'].drop_duplicates()
        daily_new = daily_new.drop(['date'], axis=1)
        daily_new['apple'] = -daily_new['apple']
        daily_new['pen'] = -daily_new['pen']
        
        files[shop][SPL].index = files[shop][SPL].date
        files[shop][SPL] = files[shop][SPL].drop(['date'], axis=1)
        
        df = pd.concat([files[shop][SPL], daily_new]).sort_index()
        df = df.resample('D').sum()
        
        l = df.index.size
        df['apple_sum'] = df['apple'].rolling(str(l) + 'D').sum()
        df['pen_sum'] = df['pen'].rolling(str(l) + 'D').sum()
        df = df.drop(['apple', 'pen'], axis=1)
        df = df.rename(columns={'apple_sum': 'apple', 'pen_sum': 'pen'})
        print(df.head())
        
        
        name = create_filenames(state, shop + 1)[DAILY]
        df.to_csv('answer/' + name, index=False)
        answer[shop] = df
        shop = shop + 1
        
    return answer
    

In [33]:
daily_b = daily_stor('b', 0)
print(daily_b)
daily_m = daily_stor('m', 1)
print(daily_m)
daily_s = daily_stor('s', 2)
print(daily_s)

['./input/MS-b1-sell.csv', './input/MS-b1-inventory.csv', './input/MS-b1-supply.csv']
['./input/MS-b2-supply.csv', './input/MS-b2-sell.csv', './input/MS-b2-inventory.csv']
sku_num                    0       1      All
date                                         
2006-01-01 00:00:00     1815     156     1971
2006-01-02 00:00:00     1862     143     2005
2006-01-03 00:00:00     1880     171     2051
2006-01-04 00:00:00     1797     153     1950
2006-01-05 00:00:00     1942     133     2075
2006-01-06 00:00:00     1898     149     2047
2006-01-07 00:00:00     1895     138     2033
2006-01-08 00:00:00     1836     120     1956
2006-01-09 00:00:00     1868     125     1993
2006-01-10 00:00:00     1879     156     2035
2006-01-11 00:00:00     1796     151     1947
2006-01-12 00:00:00     1915     140     2055
2006-01-13 00:00:00     1920     136     2056
2006-01-14 00:00:00     1872     161     2033
2006-01-15 00:00:00     1822     144     1966
2006-01-16 00:00:00     1894     160     2054


KeyError: 'sku_num'

In [48]:
def year_table():
    states = ['b', 'm', 's']
    res = {}
    t = {}
    for i in range (0, 3):
        t[i] = sold[i].resample('Y').sum()
        stolen[i] = stolen[i].resample('Y').sum()
        t[i].insert(3, 'apple_stolen',stolen[i][1])
        t[i].insert(5, 'pen_stolen',stolen[i][2])
        t[i].insert(1, 'state', states[i])
        
        
    res = pd.concat([t[0], t[1], t[2]]).sort_index()

In [46]:
def check_res():
    states = ['b', 'm', 's']
    for state in states:
        right_ansm, maxou = get_files(state, './output/*')
        our_ans, maxa = get_files(state, './answer/*') 
        for i in range (0, maxou):
            if (right_ansm[i] != our_ans[i]):
                print('ответы не сошлись')